# liste des tables dans la base de données : 


```sh
test1=# \dt
             List of relations
 Schema |       Name       | Type  | Owner 
--------+------------------+-------+-------
 public | abstractproblems | table | oscar
 public | compilers        | table | oscar
 public | courses          | table | oscar
 public | courseslists     | table | oscar
 public | coursesusers     | table | oscar
 public | listitems        | table | oscar
 public | lists            | table | oscar
 public | problems         | table | oscar
 public | problemstags     | table | oscar
 public | submissions      | table | oscar
 public | users            | table | oscar
(11 rows)
```

# Descritpion de la table des problems :

```sh
test1=# \d problems
          Table "public.problems"
        Column        |  Type   | Modifiers 
----------------------+---------+-----------
 problem_id           | text    | not null
 problem_nm           | text    | not null
 language_id          | text    | not null
 title                | text    | not null
 original_language_id | text    | not null
 checked              | integer | 
Indexes:
    "problems_pkey" PRIMARY KEY, btree (problem_id)
    "idx_problems_problem_nm" btree (problem_nm)
Foreign-key constraints:
    "problems_problem_nm_fkey" FOREIGN KEY (problem_nm) REFERENCES abstractproblems(problem_nm)
Referenced by:
    TABLE "submissions" CONSTRAINT "submissions_problem_id_fkey" FOREIGN KEY (problem_id) REFERENCES problems(problem_id)

```

## Types de probs
Il y a plusieurs types de problèmes, défférenciés par la syntaxe de leur identifiant.

* Pxxxxx
> Sont les problème officiels, écrit pas les concepteur du Juge, nous considérons qu'ils sont "bien fait" et les utiliserons comme référence.

* Xxxxxx
> le X signifie *externe". Ces problème sont créés pas les instructeurs, et ne sont validés par personne. Il sont accessibles uniquement pas les étudiants inscrits au cours de l'instructeur en question.

* Gxxxxx
> Le G signifie *Game*. Il y a très peu de problème de ce type, Il sont à ignorer completement.

* *deprecated*
> Ignorons les!


## Langues
Un problème est identifié par leur *problem_id* unique. Ce champ et en fait composé du *problem_nm* et du *language_id*.

> Il existe par exemple 3 versions du problème ***P12345***, anglais, espagnol et catalan:
* p12345_ca
* p12345_es
* p12345_en

Nous considérons une seule instance du problème pour ces 3 langues. La proportion de probs dont la solution diffère en fonction de la langue est très faible et peut être négligée. 


# Descritpion de la table des users :

```sh
test1=# \d users
                                                        Table "public.users"
    Column     |  Type   |                              Modifiers                                                 
---------------+---------+-------------------------------------------------------------------------------------
 user_id       | text    | not null default ('U'::text || "substring"(to_char(nextval('usersseq'::regclass),
               |         |                                      '00000'::text), 2, 5))
 creation_date | date    | not null default ('now'::text)::date
 administrator | integer | not null default 0
 instructor    | integer | not null default 0
 demo          | integer | not null default 0
 unregistered  | integer | default 0
Indexes:
    "users_pkey" PRIMARY KEY, btree (user_id)
Referenced by:
    TABLE "abstractproblems" CONSTRAINT "abstractproblems_user_id_fkey" FOREIGN KEY (user_id) REFERENCES users(user_id)
    TABLE "courses" CONSTRAINT "courses_user_id_fkey" FOREIGN KEY (user_id) REFERENCES users(user_id)
    TABLE "coursesusers" CONSTRAINT "coursesusers_user_id_fkey" FOREIGN KEY (user_id) REFERENCES users(user_id)
    TABLE "lists" CONSTRAINT "lists_user_id_fkey" FOREIGN KEY (user_id) REFERENCES users(user_id)
    TABLE "submissions" CONSTRAINT "submissions_user_id_fkey" FOREIGN KEY (user_id) REFERENCES users(user_id)

```

   * u00000 => user de test, testeur ** ne dois pas être comptabilisé**
   * u00001 => Jordi
   * u00002 => Salvador
   * Competition(1) ... => Jeux (user factice)
   * demo == true
   * unregistered == true

```sql
test1=# select count(*) from users where unregistered=1;
 count 
-------
    55
(1 row)

test1=# select count(*) from users where unregistered=0;
 count 
-------
 10510
(1 row)

test1=# select count(*) from users where unregistered!=0;
 count 
-------
    55
(1 row)
```

Il y a 55 users unregistred. Cela est négligeable.

 unregistred user_id 
---------
 U13594
 U16867
 U15591
 U16889
 U15803
 U15915
 U15957
 U15941
 U15970
 U16009
 U15991
 U16058
 U15913
 U16097
 U16083
 U16153
 U16303
 U15979
 U16333
 U16344
 U16359
 U16362
 U16438
 U16467
 U16488
 U16487
 U16493
 U16494
 U16490
 U16533
 U16526
 U16539
 U16580
 U16592
 U16584
 U16630
 U16683
 U16703
 U16715
 U16743
 U16748
 U16773
 U16753
 U14987
 U10576
 U01151
 U11327
 U12985
 U14450
 U14942
 U14770
 U15073
 U15202
 U15253
 U10002
(55 rows)





# Descritpion de la table des submissions :

```sql
test1=# \d submissions
                                                                         Table "public.submissions"
        Column        |              Type              |                                                     Modifiers                                                      
----------------------+--------------------------------+--------------------------------------------------------------------------------------------------------------------
 submission_uid       | text                           | not null default ('S'::text || "substring"(to_char(nextval('submissionsseq'::regclass), '000000000'::text), 2, 9))
 user_id              | text                           | not null
 problem_id           | text                           | not null
 submission_id        | text                           | not null
 compiler_id          | text                           | not null
 state                | text                           | not null
 time_in              | timestamp(0) without time zone | not null
 time_out             | timestamp(0) without time zone | 
 veredict             | text                           | 
 veredict_info        | text                           | 
 internal_error       | text                           | 
 legacy               | integer                        | 
 veredict_publics     | text                           | 
 ok_publics_but_wrong | integer                        | not null default 0
 score                | text                           | 
Indexes:
    "submissions_pkey" PRIMARY KEY, btree (submission_uid)
    "submissions_user_id_key" UNIQUE CONSTRAINT, btree (user_id, problem_id, submission_id)
    "cua_idx" btree (time_in DESC, submission_id DESC)
    "idx_submissions_problem_id" btree (problem_id)
    "idx_submissions_problem_time" btree (problem_id, time_in)
    "idx_submissions_time_in" btree (time_in)
    "idx_submissions_user_id" btree (user_id)
    "submissions_by_time" btree (time_in)
    "submissions_by_veredict" btree (veredict)
    "submissions_date_idx" btree (date(time_in))
    "users_idx" btree (user_id)
    "users_time_in_idx" btree (user_id, time_in)
Foreign-key constraints:
    "submissions_compiler_id_fkey" FOREIGN KEY (compiler_id) REFERENCES compilers(compiler_id)
    "submissions_problem_id_fkey" FOREIGN KEY (problem_id) REFERENCES problems(problem_id)

```

## descriptions des ***veredict***:
| Acronym |  Verdict |
|---------|--------:|
| AC      | Accepted |
| PE      | Presentation Error |
| WA      | Wrong Answer |
| IC      | Invalid Character |
| EE      | Execution Error |
| CE      | Compilation Error |
| NC      | Noncompliant Solution |
| Pending | Pending Submission |
| SE      | Setter Error |
| SC      | Scored |
| IE      | Internal Error |
| FE      | Fatal Errors |

### AC : Accepted 
**Meaning**
>Your program seems correct. That is, your program did not crash with any test case (either public or private), it was fast enough compared to the Jutge's solution, and its output for every case always matched the output of the Jutge's solution. For most problems, the match must be strict, although a few statements concede some flexibility. 

### Distribution des verdictes :